In [ ]:
from kamodo_ccmc.flythrough import SatelliteFlythrough as SF
import datetime as dt

# Set input values for RealFlight function call.
model = 'TIEGCM'
file_dir = 'C:/Users/rringuet/Kamodo_Data/TIEGCM/Uriel_Ramirez_012517_IT_1/'
dataset = 'cnofs'
start_utcts = dt.datetime(2015, 3, 18, 0).replace(tzinfo=dt.timezone.utc).timestamp()
end_utcts = dt.datetime(2015, 3, 21, 0).replace(tzinfo=dt.timezone.utc).timestamp()-1
variable_list = ['T_i'] # list of desired variable names
coord_type = 'GEO'  # GEO cartesian coordinates for the trajectory.
results = SF.RealFlight(dataset, start_utcts, end_utcts, model, file_dir, 
                        variable_list, coord_type)

# Functionalize the results
kamodo_object = SF.WO.Functionalize_SFResults(model,results)
kamodo_object

In [ ]:
# Get CINDI ion temperature data using pysat.
import pysat  # pip install pysat
import pysatNASA  # pip install pysatNASA

pysat_datadir = 'C:/Users/rringuet/Kamodo_Data/pysat_data/'
# pysat.params['data_dirs'] = pysat_datadir
pysat.utils.registry.register(['pysatNASA.instruments.cnofs_ivm'])
ivm = pysat.Instrument('cnofs', 'ivm', inst_id='', clean_level='clean')
start_download_date = dt.datetime.utcfromtimestamp(start_utcts)
stop_download_date = dt.datetime.utcfromtimestamp(end_utcts)
# ivm.download(start_download_date, stop_download_date)  # already on machine
ivm.load(date=start_download_date, end_date=stop_download_date)

In [ ]:
# Convert pandas timestamps into utc timestamps
import numpy as np
utc_time = ivm.index.values.astype(np.int64)/1e9

# Add to kamodo object.
kamodo_object = SF.WO.Functionalize_TimeSeries(utc_time, 'CINDIT_i', 'K', 
                                               ivm.data['ionTemperature'].values, 
                                               kamodo_object)
kamodo_object['DIFF'] = 'abs(CINDIT_i - T_i)'
kamodo_object

In [ ]:
# Now we can plot model and data on the same figure with the difference
kamodo_object.plot('T_i','CINDIT_i','DIFF')